In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

%matplotlib inline

In [6]:
path = '~/Desktop/kaggle/Porto/'

# 1. Read Code

In [7]:
df = pd.read_csv(path+'train.csv',index_col='id')#,nrows=100000)

In [8]:
df = df.reset_index(drop=True)

In [9]:
df.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
1,0,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
2,0,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
3,0,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0


# 2. Split features and Target

In [10]:
features = list(df.columns)
target = 'target'
features.remove(target)

In [11]:
X = np.array(df[features])
y = df[target]

# 3. Metirc Function

In [12]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
 
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [20]:
def gini_coefficient(preds,dtrain):
    y = dtrain.get_label()
    return 'gini', -gini_normalized(y,preds)

# 4. Xgboost

In [21]:
from sklearn.model_selection import KFold

In [30]:
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=4, 
                    learning_rate=0.04, 
                    n_estimators=1000, 
                    objective='binary:logistic',
                    nthread=-1, 
                    gamma=0, 
                    subsample=0.8,
                    colsample_bytree=0.8,  
                    scale_pos_weight=30, 
                    missing=None)

In [31]:
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(X):
    train_X, test_X = X[train_index], X[test_index]
    train_y, test_y = y[train_index], y[test_index]
    xgb.fit(train_X, train_y, 
            eval_set=[(train_X,train_y),(test_X,test_y)], 
            eval_metric=gini_coefficient,
            early_stopping_rounds=100,
            verbose=10)

[0]	validation_0-error:0.580019	validation_1-error:0.582774	validation_0-gini:-0.198663	validation_1-gini:-0.190952
Multiple eval metrics have been passed: 'validation_1-gini' will be used for early stopping.

Will train until validation_1-gini hasn't improved in 100 rounds.
[10]	validation_0-error:0.562667	validation_1-error:0.564871	validation_0-gini:-0.253134	validation_1-gini:-0.234274
[20]	validation_0-error:0.552012	validation_1-error:0.554011	validation_0-gini:-0.265306	validation_1-gini:-0.244614
[30]	validation_0-error:0.537214	validation_1-error:0.538386	validation_0-gini:-0.273344	validation_1-gini:-0.250521
[40]	validation_0-error:0.526421	validation_1-error:0.527483	validation_0-gini:-0.27947	validation_1-gini:-0.254332
[50]	validation_0-error:0.521159	validation_1-error:0.522587	validation_0-gini:-0.285288	validation_1-gini:-0.257831
[60]	validation_0-error:0.513313	validation_1-error:0.514691	validation_0-gini:-0.290184	validation_1-gini:-0.259926
[70]	validation_0-error

KeyboardInterrupt: 